In [1]:
import numpy as np

input_data = np.load('initial_data/function_3/initial_inputs.npy')
print(input_data.shape)
print(input_data)


(15, 3)
[[0.17152521 0.34391687 0.2487372 ]
 [0.24211446 0.64407427 0.27243281]
 [0.53490572 0.39850092 0.17338873]
 [0.49258141 0.61159319 0.34017639]
 [0.13462167 0.21991724 0.45820622]
 [0.34552327 0.94135983 0.26936348]
 [0.15183663 0.43999062 0.99088187]
 [0.64550284 0.39714294 0.91977134]
 [0.74691195 0.28419631 0.22629985]
 [0.17047699 0.6970324  0.14916943]
 [0.22054934 0.29782524 0.34355534]
 [0.66601366 0.67198515 0.2462953 ]
 [0.04680895 0.23136024 0.77061759]
 [0.60009728 0.72513573 0.06608864]
 [0.96599485 0.86111969 0.56682913]]


In [2]:
output_data = np.load('initial_data/function_3/initial_outputs.npy')
print(output_data.shape)
print(output_data)


(15,)
[-0.1121222  -0.08796286 -0.11141465 -0.03483531 -0.04800758 -0.11062091
 -0.39892551 -0.11386851 -0.13146061 -0.09418956 -0.04694741 -0.10596504
 -0.11804826 -0.03637783 -0.05675837]


In [3]:
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C, Matern


In [ ]:
n_dims = input_data.shape[1]
kernel = C(1.0) * Matern(length_scale=np.ones(n_dims), nu=2.5)

In [7]:
gp = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=10, normalize_y=True)
gp.fit(input_data, output_data)


GaussianProcessRegressor(kernel=1**2 * Matern(length_scale=1, nu=2.5),
                         n_restarts_optimizer=10, normalize_y=True)

In [8]:
from scipy.stats import norm

def expected_improvement(X_candidates, gp, y_best, xi=0.01):
    mu, sigma = gp.predict(X_candidates, return_std=True)
    Z = (mu - y_best - xi) / (sigma + 1e-9)
    ei = (mu - y_best - xi) * norm.cdf(Z) + sigma * norm.pdf(Z)
    return ei

# Generate candidate points and find best
candidates = np.random.rand(1000, 2)
ei_scores = expected_improvement(candidates, gp, y_best=output_data.max())
best_next = candidates[np.argmax(ei_scores)]
print(f"Suggested next input: {best_next}")

Suggested next input: [0.75904807 0.96537508]


In [10]:
mean, std = gp.predict([best_next], return_std=True)

print(f"Predicted mean: {mean[0]:.6e}")
print(f"Uncertainty (std): {std[0]:.6e}")

Predicted mean: 5.928535e-01
Uncertainty (std): 1.303603e-01
